# LSTMレイヤの実装

In [1]:
import numpy as np

try:
    from google.colab import files
    print('Google Colab. 上での実行です')
    print('「ファイルを選択」から、notebook/commonフォルダのfunctions.pyを選択し、アップロードしてください')
    print('===========')
    files.upload()
    !mkdir common
    !mv *.py ./common
except:
    print('ローカル環境での実行です')

from common.functions import sigmoid

Google Colab. 上での実行です
「ファイルを選択」から、notebook/commonフォルダのfunctions.pyを選択し、アップロードしてください


Saving functions.py to functions.py


## [演習]
* 以下のLSTMレイヤのクラスを完成させましょう

In [2]:
class LSTM:
    def __init__(self, Wx, Wh, b):
        '''
        Parameters
        ----------
        Wx: 入力x用の重みパラメータ（4つ分の重みをまとめたもの)
        Wh: 隠れ状態h用の重みパラメータ（4つ分の重みをまとめたもの）
        b: バイアス（4つ分のバイアスをまとめたもの）
        '''
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None

    def forward(self, x, h_prev, c_prev):
        """
        順伝播計算
        """        
        Wx, Wh, b = self.params
        N, H = h_prev.shape

        A = np.dot(x, Wx) + np.dot(h_prev, Wh) + b

        f = A[:, :H]
        g = A[:, H:2*H]
        i = A[:, 2*H:3*H]
        o = A[:, 3*H:]

        f = sigmoid(f)
        g = np.tanh(g)
        i = sigmoid(i)
        o = sigmoid(o)
    
#         print(f.shape, c_prev.shape, g.shape, i.shape)
        c_next = f * c_prev + g * i
        tanh_c_next = np.tanh(c_next)
        h_next = o * tanh_c_next

        self.cache = (x, h_prev, c_prev, i, f, g, o, tanh_c_next)
        return h_next, c_next

    def backward(self, dh_next, dc_next):
        """
        逆伝播計算
        """        
        Wx, Wh, b = self.params
        x, h_prev, c_prev, i, f, g, o, tanh_c_next = self.cache

        A2 = (dh_next * o) * (1 - tanh_c_next ** 2)
        ds = dc_next + A2

        dc_prev = ds * f

        di = ds * g
        df = ds * c_prev
        do = dh_next * tanh_c_next
        dg = ds * i

        di *= i * (1 - i)
        df *= f * (1 - f)
        do *= o * (1 - o)
        dg *= (1 - g ** 2)

        dA = np.hstack((df, dg, di, do))

        dWh = np.dot(h_prev.T, dA)
        dWx = np.dot(x.T, dA)
        db = dA.sum(axis=0)

        self.grads[0][:] = dWx # 同じメモリ位置に代入
        self.grads[1][:] = dWh # 同じメモリ位置に代入
        self.grads[2][:] = db # 同じメモリ位置に代入

        dx = np.dot(dA, Wx.T)
        dh_prev = np.dot(dA, Wh.T)

        return dx, dh_prev, dc_prev

In [3]:
D = 10 # 入力データの次元
H = 5 # 中間層のノード数

Wx = (np.random.randn(D, 4 * H) / np.sqrt(D))
Wh = (np.random.randn(H, 4 * H) / np.sqrt(H))
b = np.zeros(4 * H)

# オブジェクトの生成
lstm = LSTM(Wx, Wh, b)

# 順伝播計算
N = 4 # バッチサイズ
x = np.random.randn(N, D)
h_prev = np.random.randn(N, H)
c_prev = np.zeros((N, H))
h_next = lstm.forward(x, h_prev, c_prev)
print("h_next=", h_next)
print()

# 逆伝播計算
dh_next = np.random.randn(N, H)
dc_next = np.random.randn(N, H)
dx, dh_prev, dc_prev = lstm.backward(dh_next, dc_next)
print("dx=", dx)
print()
print("dh_prev=", dh_prev)
print()


h_next= (array([[ 0.30215128,  0.12860912, -0.03240782,  0.18366052, -0.23760543],
       [-0.00856664, -0.03519716,  0.08634908,  0.00513652,  0.0301568 ],
       [-0.33462764, -0.06838212, -0.11275526,  0.15037347,  0.56782379],
       [ 0.04564709,  0.46660203,  0.12126597, -0.1168428 , -0.13156511]]), array([[ 0.44692996,  0.30274918, -0.18250965,  0.26799725, -0.31690678],
       [-0.03564554, -0.09740465,  0.25906825,  0.13397995,  0.55507622],
       [-0.37701265, -0.57332244, -0.44307623,  0.55891764,  0.78206354],
       [ 0.26684797,  0.62797354,  0.42063032, -0.25763327, -0.22249913]]))

dx= [[-0.37002383  0.01293486  0.26633697  0.19988117  0.35235982 -0.29853428
  -0.18305811  0.146531   -1.12855983 -0.4806945 ]
 [-0.08668002  0.15004973  0.09910925 -0.07942835 -0.04366418 -0.01416063
   0.0638922  -0.14039869  0.09021982  0.10984481]
 [ 0.05846756  0.19114055 -0.05989803  0.06844916 -0.00537126  0.01472662
   0.06423788  0.11302348  0.06053746  0.1061408 ]
 [ 0.05529605  